In [25]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [17]:
df_green = df_green\
    .withColumnRenamed('lpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [19]:
df_yellow = df_yellow\
    .withColumnRenamed('tpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [22]:
common_columns = []
yellow_columns =  set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [29]:
df_green_sel = df_green\
    .select(common_columns)\
    .withColumn('service_type', F.lit('green'))

In [31]:
df_yellow_sel = df_yellow\
    .select(common_columns)\
    .withColumn('service_type', F.lit('yellow'))

In [32]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [34]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [37]:
df_trips_data.registerTempTable('trips_data')

/home/MERHAWI/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [51]:
df_result = spark.sql(""" 
SELECT   
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime")  AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM 
    trips_data
GROUP BY 
    1, 2, 3
""")

In [56]:
df_result.write.parquet('data/report/revenue', mode='overwrite')